In [1]:
import math

import scipy

import numpy as np

R = PolynomialRing(QQ,30,"X")

z = R.gens()

In [2]:

##Definitions polynomials.

#We represent here a polynomial P by a list L (called "polynomial list") of lists of the form [c,E], where c is a real number and 
# E is a list of positive integers. Each list of the form [c,E] in L corresponds to the monomial in P with coefficient c
# and whose variables are the one with indices the elements of E, with multiplicities.
# For instance, the list [2,[2,2,3]] corresponds to the monomial 2X_2X_2X_3.

def product(k, L): 

# take a polynomial list L representing a polynomial P and an integer k, and gives back the polynomial list 
# representing the multiplication of P with the monomial X_k.
    S = list(L)
    
    for E in S: 
        
        E[1] = sorted(E[1] + [k])
    
    return(S)

def  scalaire(c,L): 
    
# take a polynomial list L representing a polynomial P and an real number c, and gives back the polynomial list 
# representing the multiplication of P with the number c.
    
    S = list(L)
    
    for E in S:
        
        E[0] = c*E[0]
        
    return(S)

def D(L): 

# take a polynomial list representing a polynomial P and gives back the polynomial list representing the
# polynomial D(P) (defined in Definition 3.2).
    
    S = []
    
    for E in L:
        
        if  E[0] != 0:
        
            S = S + Dintermed(E)
        
    return(S) 

def Dintermed(E):
    
# auxiliary function for D.
    
    L = []
    
    l = len(E[1])
    
    for i in range(l):
        
        W = list(E[1])
        
        W[i] = W[i] + 1
        
        L = L + [[E[0],sorted(W)]]
        
    return L 

def D2(L): 

# take a polynomial list representing a polynomial P and gives back the polynomial list representing the
# polynomial D_2(P) (defined in Definition 3.2).
    
    S = []
    
    for E in L:
        
        if E[0] != 0 :
        
            S = S + Dintermed2(E)
        
    return(S)

def Dintermed2(E):

# auxiliary function for D2.
    
    rep = []
    
    l = len(E[1])
    
    for i in range(l):
        
        W = list(E[1])
        
        W[i] = W[i] + 2
        
        rep = rep + [[-E[0]/2,sorted(W)]]
        
    return rep 



def H(L):

# take a polynomial list representing a polynomial P and gives back the polynomial list representing the
# polynomial H(P) (defined in Definition 3.3).
    
    S = []
    
    for E in L:
        
        if E[0] != 0:
        
            for j in range(len(E[1])):
            
                auxbis = []
                    
                for i in range(E[1][j]+1):
                    
                    auxbis = auxbis + [[-(1/2)*binomial(E[1][j],i)*E[0], [i+1,1+E[1][j]-i]]]
                    
                    
                for F in auxbis:
                
                        F[1] = sorted(F[1] + Hintermed1(j,E[1]))
                
                S = S + auxbis
    
    return S

def Hintermed1(k,lst): 
    
# where k <= len(lst) ; auxiliary function for H.
    
    rep = []
    
    n = len(lst)
    
    for i in range(n):
        
        if i != k :
            
            rep = rep + [lst[i]]
    
    return rep


##Algorithms to calculate the partition of an integer. 

def successeur(a):
    
    k = len(a)
    
    if k==1:
        
        return a
    
    y = a[k-1]-1
    
    x = a[k-2]+1
    
    q = y//x
    
    b = a[:k-2]+q*[x]+[y-(q-1)*x]
    
    return b

def partitions(n):
    
    a = n*[1]
    
    L = [a]
    
    while a!=[n]:
        
        a = successeur(a)
        
        L += [a]
    
    return L

def partitionR(n): 
    
#calculates the partitions appearing for the sum in the polynomial R_n.
    
    L = partitions(2*n)
    
    S = []
    
    for k in L:
        
        if not ap1(k):
            
            if k[-1] <= n-1:
                
                if len(k) <= n:
            
                    S.append(k)
    
    
    
    return S

def ap1(k):
    
    rep = False
    
    for i in k:
        
        if i==1 :
            
            return True
    
    return rep

##Algorithms to compute R_n.

def initial(n): 
    
#list of the monomials appearing in R_n.
    
    L = partitionR(n)
    
    S = []
    
    for k in L:
        
        S.append([0,k])
        
    return S

def Rbis(n): 
    
#compute R_n
    
    rep = [[-2,[2,2,2]]]
    
    if n == 3: return rep
    
    for k in range(3,n): # n is greater than or equal to 4.
        
        aux = []
        
        S = initial(k+1)
        
        m = len(S)
        
        #compute A_n
        
        for i in range(1,k):
            
            aux.append([-binomial(k,i),sorted([i+1,k,k+1-i])])
        
        ajout(aux,S,m,k)
        
        #compute [\Delta,V_n]
        
        aux = scalaire(1/2,D(D(rep))) + D2(rep)
        
        ajout(aux,S,m,k)
        
        #compute [\Gamma,V_n]
        
        aux = []
        
        aux = product(1,D(rep)) + H(rep)
        
        ajout(aux,S,m,k)
        
        #setting induction
        
        rep = S
        
    return rep

def ajout(L,S,m,k): 
    
# auxiliary function.
    
    for C in L:
        
        i = 0
        
        boue = True
        
        if C[1][-1] == k+1 or C[1][0] == 1:
            
            boue = False
        
        while boue and i<m and C[1] != S[i][1]:
            
            i = i + 1
            
        if boue == True:
            
            S[i][0] = S[i][0] + C[0]
 
            
def convert_aux(E): 
    
# auxiliary function for convert.
    
    a = E[0]
    
    b = E[1]
    
    P = 1
    
    for i in b:
        
        P = P*z[i]
        
    return a*P

def convert(L): 
    
# converts a polynomial list L into the corresponding polynomial.
    
    P = 0
    
    for E in L:
        
        P = P + convert_aux(E)
    
    return P 


    
def R_tilde(n): 
    
# We only add X_l^2 to R_l ; computes Rtilda_n.
    
    rep = Rbis(n) + [[1,[n,n]]]
    
    #rep = convert(rep)
    
    return rep

def R_nonzero(n): 
    
# remove monomial with odd indices in R_n.
    
    L = R_tilde(n)
    
    S = []
    
    for k in L:
        
        if k[0] != 0 and no_odd(k[1]):
            
            S.append(k)
       
    return convert(S)
        
def no_odd(L): 
    
# auxiliary function for R_nonzero.
        
    rep = True
        
    for k in L:
            
        if k%2 != 0:
                
            return False
            
    return rep

def aN(n): 
    
# compute a_n.
    
    aux = R_tilde(n+1)
    
    S = []
    
    for E in aux:
        
        if E[0] != 0 and E[1][-1] == n:
            
            S.append(E)
        
        
    S = S[1:]
    
    for E in S:
        
        E[1] = E[1][:-1]
    
    return S

def aN1(n):

# compute a_n.
    
    aux = R_tilde(n+1)
    
    S = []
    
    for E in aux:
        
        if E[0] != 0 and E[1][-1] == n-1:
            
            S.append(E)
        
        
    S = S[1:]
    
    for E in S:
        
        E[1] = E[1][:-1]
    
    return S


def P(n): #n>
    
    S = [[1,[1]]]
    
    for k in range(1,n+1):
        
        aux = partP(k+1)
        
        m = len(aux)
        
        aux1 = D(S)
        
        
        for E in aux1:
            
            E[1] = sorted(E[1]+[0 for i in range(2**(k-1))])
        
        
        ajoutP(aux1,aux,m,k+1)
        
            
        for E in S:
            
            E[0] = -(2^(k-1))*E[0]
            
            E[1] = sorted(E[1]+ [1] +[0 for i in range(2**(k-1)-1)])
            
            
        ajoutP(S,aux,m,k+1)
            
        S = aux
        
    return S

def T(n):
    
    aux = P(n)
    
    for E in aux:
        
        E0 = []
        
        for k in E[1]:
            
            if k != 0:
                
                E0.append(k)
                
            E[1] = E0
        
    return aux
        
def partP(n): 
    
#list of the monomials appearing in R_n.
    
    L = partitions(n)
    
    S = []
    
    for k in L:
        
        s = lasomme(k)
        
        S.append([0,[0 for i in range(2**(n-1)-len(k))]+k])
        
    return S

def lasomme(L):
    
    s = 0
    
    for k in L:
        
        s+=k
        
    return(s)

def ajoutP(L,S,m,k): 
    
# auxiliary function.
    
    for C in L:
        
        i = 0
        
        boue = True
        
        while i<m and C[1] != S[i][1]:
            
            i = i + 1
            
        S[i][0] = S[i][0] + C[0]
        
        
        
        

In [3]:
L = []
for k in range(3,21):
    
    P = Rbis(k)
    
    L.append(P)
    
    print(" ")
    
    print(" ")
    
    print('R%s'%k + " = %s"%convert(P))
    
          


 
 
R3 = -2*X2^3
 
 
R4 = 6*X2^4 - 12*X2*X3^2
 
 
R5 = -24*X2^5 + 120*X2^2*X3^2 - 30*X3^2*X4 - 20*X2*X4^2
 
 
R6 = 120*X2^6 - 1200*X2^3*X3^2 + 210*X3^4 + 900*X2*X3^2*X4 + 300*X2^2*X4^2 - 30*X4^3 - 120*X3*X4*X5 - 30*X2*X5^2
 
 
R7 = -720*X2^7 + 12600*X2^4*X3^2 - 8820*X2*X3^4 - 18900*X2^2*X3^2*X4 - 4200*X2^3*X4^2 + 4830*X3^2*X4^2 + 1260*X2*X4^3 + 1260*X3^3*X5 + 5040*X2*X3*X4*X5 + 630*X2^2*X5^2 - 210*X4*X5^2 - 140*X4^2*X6 - 210*X3*X5*X6 - 42*X2*X6^2
 
 
R8 = 5040*X2^8 - 141120*X2^5*X3^2 + 246960*X2^2*X3^4 + 352800*X2^3*X3^2*X4 + 58800*X2^4*X4^2 - 95760*X3^4*X4 - 270480*X2*X3^2*X4^2 - 35280*X2^2*X4^3 - 70560*X2*X3^3*X5 - 141120*X2^2*X3*X4*X5 - 11760*X2^3*X5^2 + 6230*X4^4 + 40320*X3*X4^2*X5 + 14280*X3^2*X5^2 + 11760*X2*X4*X5^2 + 11760*X3^2*X4*X6 + 7840*X2*X4^2*X6 + 11760*X2*X3*X5*X6 + 1176*X2^2*X6^2 - 560*X5^2*X6 - 420*X4*X6^2 - 560*X4*X5*X7 - 336*X3*X6*X7 - 56*X2*X7^2
 
 
R9 = -40320*X2^9 + 1693440*X2^6*X3^2 - 5927040*X2^3*X3^4 - 6350400*X2^4*X3^2*X4 - 846720*X2^5*X4^2 + 534240*X3^6 + 6894

 
 
R13 = -479001600*X2^13 + 68497228800*X2^10*X3^2 - 1438441804800*X2^7*X3^4 - 770593824000*X2^8*X3^2*X4 - 57081024000*X2^9*X4^2 + 4537941408000*X2^4*X3^6 + 11713026124800*X2^5*X3^4*X4 + 5514026918400*X2^6*X3^2*X4^2 + 205491686400*X2^7*X4^3 + 1438441804800*X2^6*X3^3*X5 + 821966745600*X2^7*X3*X4*X5 + 25686460800*X2^8*X5^2 - 1002161160000*X2*X3^8 - 12971662704000*X2^2*X3^6*X4 - 30923644752000*X2^3*X3^4*X4^2 - 14833931112000*X2^4*X3^2*X4^3 - 762031670400*X2^5*X4^4 - 6404490892800*X2^3*X3^5*X5 - 16824631824000*X2^4*X3^3*X4*X5 - 4931800473600*X2^5*X3*X4^2*X5 - 1746679334400*X2^5*X3^2*X5^2 - 239740300800*X2^6*X4*X5^2 - 899026128000*X2^4*X3^4*X6 - 1438441804800*X2^5*X3^2*X4*X6 - 159826867200*X2^6*X4^2*X6 - 239740300800*X2^6*X3*X5*X6 - 6849722880*X2^7*X6^2 + 3688861176000*X3^6*X4^2 + 18158252112000*X2*X3^4*X4^3 + 13535337816000*X2^2*X3^2*X4^4 + 887609923200*X2^3*X4^5 + 602464262400*X3^7*X5 + 12874365504000*X2*X3^5*X4*X5 + 32450562144000*X2^2*X3^3*X4^2*X5 + 11016637632000*X2^3*X3*X4^3*X5 + 519

 
 
R15 = -87178291200*X2^15 + 19833061248000*X2^12*X3^2 - 763572858048000*X2^9*X3^4 - 327245510592000*X2^10*X3^2*X4 - 19833061248000*X2^11*X4^2 + 5781337353792000*X2^6*X3^6 + 10658853773568000*X2^7*X3^4*X4 + 3763323371808000*X2^8*X3^2*X4^2 + 109081836864000*X2^9*X4^3 + 981736531776000*X2^8*X3^3*X5 + 436327347456000*X2^9*X3*X4*X5 + 10908183686400*X2^10*X5^2 - 6383766589200000*X2^3*X3^8 - 41314745712240000*X2^4*X3^6*X4 - 59095085121072000*X2^5*X3^4*X4^2 - 18898428236688000*X2^6*X3^2*X4^3 - 693448820064000*X2^7*X4^4 - 12238982096140800*X2^5*X3^5*X5 - 21434580943776000*X2^6*X3^3*X4*X5 - 4487938430976000*X2^7*X3*X4^2*X5 - 1589478194304000*X2^7*X3^2*X5^2 - 163622755296000*X2^8*X4*X5^2 - 1145359287072000*X2^6*X3^4*X6 - 1308982042368000*X2^7*X3^2*X4*X6 - 109081836864000*X2^8*X4^2*X6 - 163622755296000*X2^8*X3*X5*X6 - 3636061228800*X2^9*X6^2 + 238636950552000*X3^10 + 10320819949440000*X2*X3^8*X4 + 70494137073360000*X2^2*X3^6*X4^2 + 115668065953440000*X2^3*X3^4*X4^3 + 43110050943960000*X2^4*X3^2

 
 
R16 = 1307674368000*X2^16 - 366148823040000*X2^13*X3^2 + 18325748593152000*X2^10*X3^4 + 7139902049280000*X2^11*X3^2*X4 + 396661224960000*X2^12*X4^2 - 198217280701440000*X2^7*X3^6 - 319765613207040000*X2^8*X3^4*X4 - 100355289914880000*X2^9*X3^2*X4^2 - 2617964084736000*X2^10*X4^3 - 26179640847360000*X2^9*X3^3*X5 - 10471856338944000*X2^10*X3*X4*X5 - 237996734976000*X2^11*X5^2 + 383025995352000000*X2^4*X3^8 + 1983107794187520000*X2^5*X3^6*X4 + 2363803404842880000*X2^6*X3^4*X4^2 + 647946110972160000*X2^7*X3^2*X4^3 + 20803464601920000*X2^8*X4^4 + 489559283845632000*X2^6*X3^5*X5 + 734899918072320000*X2^7*X3^3*X4*X5 + 134638152929280000*X2^8*X3*X4^2*X5 + 47684345829120000*X2^8*X3^2*X5^2 + 4363273474560000*X2^9*X4*X5^2 + 39269461271040000*X2^7*X3^4*X6 + 39269461271040000*X2^8*X3^2*X4*X6 + 2908848983040000*X2^9*X4^2*X6 + 4363273474560000*X2^9*X3*X5*X6 + 87265469491200*X2^10*X6^2 - 57272868132480000*X2*X3^10 - 1238498393932800000*X2^2*X3^8*X4 - 5639530965868800000*X2^3*X3^6*X4^2 - 69400839572

 
 
R17 = -20922789888000*X2^17 + 7113748561920000*X2^14*X3^2 - 453145783394304000*X2^11*X3^4 - 161837779783680000*X2^12*X3^2*X4 - 8299373322240000*X2^13*X4^2 + 6739387543848960000*X2^8*X3^6 + 9664027421368320000*X2^9*X3^4*X4 + 2729663885684736000*X2^10*X3^2*X4^2 + 64735111913472000*X2^11*X4^3 + 712086231048192000*X2^10*X3^3*X5 + 258940447653888000*X2^11*X3*X4*X5 + 5394592659456000*X2^12*X5^2 - 20836614147148800000*X2^5*X3^8 - 89900886669834240000*X2^6*X3^6*X4 - 91850646588180480000*X2^7*X3^4*X4^2 - 22030167773053440000*X2^8*X3^2*X4^3 - 628726930191360000*X2^9*X4^4 - 19022875029430272000*X2^7*X3^5*X5 - 24986597214458880000*X2^8*X3^3*X4*X5 - 4069064177418240000*X2^9*X3*X4^2*X5 - 1441126896168960000*X2^9*X3^2*X5^2 - 118681038508032000*X2^10*X4*X5^2 - 1335161683215360000*X2^8*X3^4*X6 - 1186810385080320000*X2^9*X3^2*X4*X6 - 79120692338688000*X2^10*X4^2*X6 - 118681038508032000*X2^10*X3*X5*X6 - 2157837063782400*X2^11*X6^2 + 7789110066017280000*X2^2*X3^10 + 112290521049907200000*X2^3*X3^8*X4 

 
 
R18 = 355687428096000*X2^18 - 145120470663168000*X2^15*X3^2 + 11555217476554752000*X2^12*X3^4 + 3809412354908160000*X2^13*X3^2*X4 + 181400588328960000*X2^14*X4^2 - 229139176490864640000*X2^9*X3^6 - 295719239093870592000*X2^10*X3^4*X4 - 75934286274502656000*X2^11*X3^2*X4^2 - 1650745353793536000*X2^12*X4^3 - 19808944245522432000*X2^11*X3^3*X5 - 6602981415174144000*X2^12*X3*X4*X5 - 126980411830272000*X2^13*X5^2 + 1062667321504588800000*X2^6*X3^8 + 3929953045852753920000*X2^7*X3^6*X4 + 3513287231997903360000*X2^8*X3^4*X4^2 + 749025704283816960000*X2^9*X3^2*X4^3 + 19239044063855616000*X2^10*X4^4 + 727624969875707904000*X2^8*X3^5*X5 + 849544305291601920000*X2^9*X3^3*X4*X5 + 124513363828998144000*X2^10*X3*X4^2*X5 + 44098483022770176000*X2^10*X3^2*X5^2 + 3301490707587072000*X2^11*X4*X5^2 + 45395497229322240000*X2^9*X3^4*X6 + 36316397783457792000*X2^10*X3^2*X4*X6 + 2200993805058048000*X2^11*X4^2*X6 + 3301490707587072000*X2^11*X3*X5*X6 + 55024845126451200*X2^12*X6^2 - 794489226733762560000*X

 
 
R19 = -6402373705728000*X2^19 + 3101950060425216000*X2^16*X3^2 - 303991105921671168000*X2^13*X3^4 - 93058501812756480000*X2^14*X3^2*X4 - 4135933413900288000*X2^15*X4^2 + 7836559835987570688000*X2^10*X3^6 + 9194179979100340224000*X2^11*X3^4*X4 + 2164127158823325696000*X2^12*X3^2*X4^2 + 43427300845953024000*X2^13*X4^3 + 564554910997389312000*X2^12*X3^3*X5 + 173709203383812096000*X2^13*X3*X4*X5 + 3101950060425216000*X2^14*X5^2 - 51918889136367052800000*X2^7*X3^8 - 168005492710205230080000*X2^8*X3^6*X4 - 133504914815920327680000*X2^9*X3^4*X4^2 - 25616679086506540032000*X2^10*X3^2*X4^3 - 598159369985329152000*X2^11*X4^4 - 27649748855276900352000*X2^9*X3^5*X5 - 29054415240972785664000*X2^10*X3^3*X4*X5 - 3871233675410669568000*X2^11*X3*X4^2*X5 - 1371061926707945472000*X2^11*X3^2*X5^2 - 94092485166231552000*X2^12*X4*X5^2 - 1552526005242820608000*X2^10*X3^4*X6 - 1129109821994778624000*X2^11*X3^2*X4*X6 - 62728323444154368000*X2^12*X4^2*X6 - 94092485166231552000*X2^12*X3*X5*X6 - 1447576694865

 
 
R20 = 121645100408832000*X2^20 - 69337707233034240000*X2^17*X3^2 + 8251187160731074560000*X2^14*X3^4 + 2357482045923164160000*X2^15*X3^2*X4 + 98228418580131840000*X2^16*X4^2 - 270717521606843351040000*X2^11*X3^6 - 291149032671510773760000*X2^12*X3^4*X4 - 63259101565604904960000*X2^13*X3^2*X4^2 - 1178741022961582080000*X2^14*X4^3 - 16502374321462149120000*X2^13*X3^3*X5 - 4714964091846328320000*X2^14*X3*X4*X5 - 78582734864105472000*X2^15*X5^2 + 2466147233977435008000000*X2^8*X3^8 + 7093565247764220825600000*X2^9*X3^6*X4 + 5073186763004972451840000*X2^10*X3^4*X4^2 + 884939822988407746560000*X2^11*X3^2*X4^3 + 18941713382868756480000*X2^12*X4^4 + 1050690456500522213376000*X2^10*X3^5*X5 + 1003697981051787141120000*X2^11*X3^3*X4*X5 + 122589066388004536320000*X2^12*X3*X4^2*X5 + 43416961012418273280000*X2^12*X3^2*X5^2 + 2750395720243691520000*X2^13*X4*X5^2 + 53632716544751984640000*X2^11*X3^4*X6 + 35755144363167989760000*X2^12*X3^2*X4*X6 + 1833597146829127680000*X2^13*X4^2*X6 + 275039572024

In [11]:
n = []
for liste in L:
    
    n.append(len(liste))


In [12]:
n

[1,
 2,
 5,
 10,
 19,
 33,
 58,
 95,
 154,
 243,
 377,
 573,
 863,
 1276,
 1870,
 2709,
 3888,
 5526]